In [57]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def get_total_pages(base_url, catalog_type):
    """
    Fetch the first page and parse the pagination links
    to compute how many pages are available for this catalog.
    """
    url = f"{base_url}/solutions/products/product-catalog/?start=0&type={catalog_type}"
    resp = requests.get(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    # Find all the numbered page links
    pages = [int(a.text) for a in soup.select('a.pagination__link') if a.text.isdigit()]
    return max(pages) if pages else 1

def scrape_catalog(catalog_type, catalog_name, base_url):
    """
    Loop through every page for a given catalog_type (1 or 2),
    scrape all rows, and tag them with the correct catalog_name.
    """
    total = get_total_pages(base_url, catalog_type)
    assessments = []

    # Decide which selector to use for rows
    row_selector = {
        2: 'tr[data-course-id]',      # Pre‑packaged Job Solutions
        1: 'tr[data-entity-id]'       # Individual Test Solutions
    }[catalog_type]

    for page in range(total):
        start = page * 12
        url = f"{base_url}/solutions/products/product-catalog/?start={start}&type={catalog_type}"
        resp = requests.get(url)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, 'html.parser')

        for row in soup.select(row_selector):
            title_cell = row.find('td', class_='custom__table-heading__title')
            if not title_cell:
                continue
            name = title_cell.get_text(strip=True)
            link_tag = row.find('a', href=True)
            link = base_url + link_tag['href'] if link_tag else None
            # These two look at the same indicator span; adjust selectors if needed
            remote_testing = 'Yes' if row.find('span', class_='catalogue__circle -yes') else 'No'
            adaptive_irt   = 'Yes' if row.find('span', class_='catalogue__circle -yes') else 'No'
            test_type = ''.join([s.get_text(strip=True) for s in row.select('span.product-catalogue__key')])

            assessments.append({
                'name': name,
                'url': link,
                'remote_testing': remote_testing,
                'adaptive_irt': adaptive_irt,
                'test_type': test_type,
                'catalog': catalog_name
            })

        time.sleep(1)  # be polite to the server

    return assessments

if __name__ == "__main__":
    base = "https://www.shl.com"
    # Scrape both catalogues separately
    prepack = scrape_catalog(2, "Pre-packaged Job Solutions", base)
    indiv  = scrape_catalog(1, "Individual Test Solutions", base)

    # Combine & dedupe on URL
    df = pd.DataFrame(prepack + indiv).drop_duplicates(subset=["url"])

    # Save to Excel
    df.to_excel("assessments.xlsx", index=False)
    print("Saved", len(df), "unique assessments to assessments.xlsx")


Saved 518 unique assessments to assessments.xlsx


In [59]:
import re
from tqdm import tqdm

# Load your existing Excel file
df = pd.read_excel("assessments.xlsx")  # Replace with your file name
url_column = "url"

# Function to extract data from SHL page
def extract_details(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    def extract_text(h4_title):
        try:
            return soup.find('h4', string=h4_title).find_next('p').text.strip()
        except:
            return None
    
    def get_duration():
        raw = extract_text("Assessment length")
        if raw:
            match = re.search(r'\d+', raw)
            return match.group() if match else None
        return None

    return {
        "assessment_title": soup.find("h1").text.strip() if soup.find("h1") else None,
        "description": extract_text("Description"),
        "job_level": extract_text("Job levels"),
        "language": extract_text("Languages"),
        "duration_minutes": get_duration(),
        "test_types_extracted": ", ".join([el.text for el in soup.select('.product-catalogue__key')]),
        "remote_indicator": "Yes" if soup.select_one('.catalogue__circle.-yes') else "No"
    }

# Iterate over each URL and extract new info
scraped_data = []
for url in tqdm(df[url_column], desc="Scraping SHL links"):
    try:
        res = requests.get(url)
        if res.status_code == 200:
            details = extract_details(res.text)
        else:
            details = {k: None for k in ["assessment_title", "description", "job_level", "language", "duration_minutes", "test_types_extracted", "remote_indicator"]}
    except Exception as e:
        print(f"Error processing {url}: {e}")
        details = {k: None for k in ["assessment_title", "description", "job_level", "language", "duration_minutes", "test_types_extracted", "remote_indicator"]}
    scraped_data.append(details)

# Merge new data with existing dataframe
scraped_df = pd.DataFrame(scraped_data)
final_df = pd.concat([df, scraped_df], axis=1)

# Save to updated Excel
final_df.to_excel("updated_shl_data.xlsx", index=False)
print("✅ Done. Data saved to 'updated_shl_data.xlsx'")



Scraping SHL links: 100%|█████████████████████| 518/518 [11:04<00:00,  1.28s/it]

✅ Done. Data saved to 'updated_shl_data.xlsx'


In [65]:
scraped_df.head()

,assessment_title,description,job_level,language,duration_minutes,test_types_extracted,remote_indicator,search_text
0,Account Manager Solution,The Account Manager solution is an assessment ...,"Mid-Professional,","English (USA),",49,"C, P, A, B, A, B, C, D, E, K, P, S",Yes,Account Manager Solution The Account Manager s...
1,Administrative Professional - Short Form,The Administrative Professional solution is fo...,"Entry-Level,","English (USA),",36,"A, K, P, A, B, C, D, E, K, P, S",Yes,Administrative Professional - Short Form The A...
2,Agency Manager Solution,The Agency Manager solution is for mid-level s...,"Front Line Manager, Manager, Supervisor,","English (USA),",51,"A, B, P, S, A, B, C, D, E, K, P, S",Yes,Agency Manager Solution The Agency Manager sol...
3,Apprentice + 8.0 Job Focused Assessment,The Apprentice + 8.0 Job-Focused Assessment is...,"General Population, Graduate, Entry-Level,","English International, German,",30,"B, P, A, B, C, D, E, K, P, S",Yes,Apprentice + 8.0 Job Focused Assessment The Ap...
4,Apprentice 8.0 Job Focused Assessment,The Apprentice 8.0 Job-Focused Assessment is a...,"Entry-Level, General Population, Graduate,","English International, German, French,",20,"B, P, A, B, C, D, E, K, P, S",Yes,Apprentice 8.0 Job Focused Assessment The Appr...


In [79]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load data
df = scraped_df

# Convert duration to numeric and handle missing values
df["duration_minutes"] = pd.to_numeric(df["duration_minutes"], errors="coerce")

# Preprocessing improvements
def clean_and_expand(text):
    text = re.sub(r",\s*", " ", str(text))
    expansions = {
        r"\b(A)\b": "Analytical",
        r"\b(B)\b": "Behavioral",
        r"\b(C)\b": "Cognitive",
        r"\b(P)\b": "Practical",
        r"\b(K)\b": "Knowledge"
    }
    for pattern, replacement in expansions.items():
        text = re.sub(pattern, replacement, text)
    return text.lower().strip()

def create_search_text(row):
    return (
        f"{clean_and_expand(row['assessment_title'])} " * 3 +
        f"{clean_and_expand(row['description'])} " * 2 +
        clean_and_expand(row['job_level']) + " " +
        clean_and_expand(row['test_types_extracted']) + " " +
        ("remote_yes " if row['remote_indicator'] == "Yes" else "remote_no ") +
        (f"duration_{int(row['duration_minutes'])} " 
         if pd.notnull(row['duration_minutes']) else "")
    )

df["search_text"] = df.apply(create_search_text, axis=1)

# TF-IDF configuration
vectorizer = TfidfVectorizer(
    stop_words="english",
    ngram_range=(1, 2),
    min_df=2,
    max_features=10000
)
tfidf_matrix = vectorizer.fit_transform(df["search_text"])

def recommend_assessments(query, top_n=5, duration_filter=None):
    processed_query = clean_and_expand(query)
    df_filtered = df.copy()
    
    if duration_filter:
        match = re.search(r"\d+", duration_filter)
        if match:
            duration = int(match.group())
            df_filtered = df_filtered[
                (df_filtered["duration_minutes"] <= duration) &
                pd.notnull(df_filtered["duration_minutes"])
            ]
        else:
            print("Could not parse duration from filter. Ignoring duration constraint.")

    query_vec = vectorizer.transform([processed_query])
    sim_scores = cosine_similarity(query_vec, vectorizer.transform(df_filtered["search_text"])).flatten()
    
    top_indices = sim_scores.argsort()[-top_n:][::-1]
    results = df_filtered.iloc[top_indices].copy()
    results["similarity_score"] = (sim_scores[top_indices] * 100).round(1)
    
    # Formatting output
    return results[["assessment_title", "description", "job_level", 
                   "test_types_extracted", "duration_minutes", "remote_indicator",
                   "similarity_score"]].style.format({
        "description": lambda x: (x[:75] + "...") if len(x) > 75 else x,
        "similarity_score": "{:.1f}%",
        "duration_minutes": lambda x: f"{int(x)} mins" if pd.notnull(x) else "N/A"
    }).set_properties(**{
        "text-align": "left",
        "white-space": "pre-wrap"
    })

# Example usage
user_query = "entry-level manager test with leadership skills under 40 minutes"
recommendations = recommend_assessments(user_query, duration_filter="40 minutes")
display(recommendations)

,assessment_title,description,job_level,test_types_extracted,duration_minutes,remote_indicator,similarity_score
447,SHL Verify Interactive G+,SHL Verify Interactive G+ (SVIG+) is a test of general cognitive ability th...,"Graduate, Manager, Mid-Professional, Professional Individual Contributor,","A, A, B, C, D, E, K, P, S",36 mins,Yes,17.6%
159,AI Skills,The AI Skills assessment measures the skills that help candidates successfu...,"General Population,","P, A, B, C, D, E, K, P, S",16 mins,Yes,14.8%
209,Culinary Skills (New),"Multi-choice test that measures the knowledge of cooking principles, cookin...","Graduate, Manager, Mid-Professional, Professional Individual Contributor, Supervisor,","K, A, B, C, D, E, K, P, S",7 mins,Yes,11.8%
267,Global Skills Assessment,The Global Skills Assessment (GSA) is an assessment used to measure 96 disc...,None,"C, K, A, B, C, D, E, K, P, S",16 mins,Yes,11.5%
119,Store Manager 7.1 (Americas),Our Store Manager 7.1 solution is designed for candidates applying to entry...,"Front Line Manager, Manager,","B, C, A, B, C, D, E, K, P, S",34 mins,Yes,9.7%


In [73]:
user_query = " Professioanl AI "
recommendations = recommend_assessments(user_query, duration_filter="")
display(recommendations)

,assessment_title,description,job_level,test_types_extracted,duration_minutes,remote_indicator,similarity_score
159,AI Skills,The AI Skills assessment measures the skills that help candidates successfu...,"General Population,","P, A, B, C, D, E, K, P, S",16 mins,Yes,75.1%
176,Automata (New),An AI-powered coding simulation assessment that evaluates candidate’s progr...,"Mid-Professional, Professional Individual Contributor,","S, A, B, C, D, E, K, P, S",45 mins,Yes,10.6%
180,Automata Pro (New),An AI-powered coding simulation assessment that evaluates candidate’s progr...,"Mid-Professional, Professional Individual Contributor,","S, A, B, C, D, E, K, P, S",60 mins,Yes,10.3%
517,360° Multi-Rater Feedback System (MFS),"SHL 360, or the Multi-Rater Feedback System (‘MFS’), provides a holistic 36...","Director, Executive, Front Line Manager, Manager, Mid-Professional, Professional Individual Contributor,","D, P, A, B, C, D, E, K, P, S",N/A,Yes,0.0%
164,Apache Hadoop (New),"Multi-choice test that measures the knowledge of basic concepts of Hadoop, ...","Mid-Professional, Professional Individual Contributor,","K, A, B, C, D, E, K, P, S",7 mins,Yes,0.0%
